In [ ]:
import pandas as pd

df = pd.read_csv("C:/Users/H/crypto_data/crypto_data/LTC-USD.csv", names = ['time','low','high','open','close','volume'])

df.head()

In [ ]:
import pandas as pd
import os
from sklearn import preprocessing
from collections import deque
import numpy as np
import random
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint



seq_len = 60
fut_per_pre = 3
rat_to_pre = "LTC-USD"
epochs = 10
batch_size = 64
name = f"{seq_len}-SEQ-{fut_per_pre}-PRED-{int(time.time())}"

def classify(cur,fut):
    if float(fut)>float(cur):
        return 1
    else:
        return 0

def preprocess_df(df):
    df = df.drop('future',1)
    
    for col in df.columns:
        if col != 'target':
            df[col] = df[col].pct_change()
            #pct_change normalizes the values
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
        
    df.dropna(inplace = True)
    
    sequential_data = []
    prev_days = deque(maxlen=seq_len)
    
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        
        if(len(prev_days) == seq_len):
            sequential_data.append([np.array(prev_days),i[-1]])
            
    random.shuffle(sequential_data)
    
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target==0:
            sells.append([seq,target])
        elif target==1:
            buys.append([seq,target])
    
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys+sells
    
    random.shuffle(sequential_data)
    
    X = []
    Y = []
    
    for seq,tar in sequential_data:
        X.append(seq)
        Y.append(tar)
    
    return np.array(X),Y
    
main_df = pd.DataFrame()

ratios = ['BTC-USD','LTC-USD','ETH-USD','BCH-USD']

for ratio in ratios:
    dataset = f"C:/Users/H/crypto_data/crypto_data/{ratio}.csv"
    
    df = pd.read_csv(dataset,names =  ['time','low','high','open','close','volume'])
    
    df.rename(columns = {'close':f"{ratio}_close", 'volume':f"{ratio}_volume"}, inplace = True)
    
    df.set_index("time",inplace=True)
    df = df[[f"{ratio}_close",f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
        
    else:
        main_df = main_df.join(df)
        

main_df['future'] = main_df[f"{rat_to_pre}_close"].shift(-fut_per_pre)

main_df['target'] = list(map(classify,main_df[f"{rat_to_pre}_close"],main_df['future']))

times = sorted(main_df.index.values)
last_5pct = times[-int(.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]


train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

training_y = []
validating_y = []
for item in train_y:
    training_y.append([item])

for item in validation_y:
    validating_y.append([item])
    
training_y = np.array(training_y)
validating_y = np.array(validating_y)
    
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont Buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")


model = Sequential()

model.add(LSTM(128,input_shape = (train_x.shape[1:]), return_sequences = True))
model.add(Dropout(.2))
model.add(BatchNormalization())

model.add(LSTM(128,input_shape = (train_x.shape[1:]), return_sequences = True))
model.add(Dropout(.1))
model.add(BatchNormalization())

model.add(LSTM(128,input_shape = (train_x.shape[1:])))
model.add(Dropout(.2))
model.add(BatchNormalization())

model.add(Dense(32,activation = 'relu'))
model.add(Dropout(.2))

model.add(Dense(2,activation = 'softmax'))

opt = tf.keras.optimizers.Adam(lr = .001,decay = 1e-6)

model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = opt,
             metrics = ['accuracy'])

tensorboard = TensorBoard(log_dir = f'C:\\Users\\H\\logs\\{name}')

filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"
checkpoint = ModelCheckpoint("C:/Users/H/Documents/{}.model".format(filepath,monitor = 'val_acc',verbose = 1,save_best_only =True,mode = 'max'))
 
history = model.fit(train_x,training_y,batch_size=batch_size,epochs = epochs,validation_data = (validation_x,validating_y),callbacks = [tensorboard,checkpoint])

